In [2]:
import rasterio
import rioxarray

In [3]:
tif_pre_fire = "../rasters/lansat/2023_valpo_swir16-nir-red.tif"
tif_post_fire = "../rasters/lansat/2024_valpo_swir16-nir-red.tif"

pre_fire = rasterio.open(tif_pre_fire)
post_fire = rasterio.open(tif_post_fire)

In [4]:
# Usamos las bandas SWIR y NIR para calcular el NBR
nbr_pre_fire = (pre_fire.read(1) - pre_fire.read(2)) / (pre_fire.read(1) + pre_fire.read(2))
nbr_post_fire = (post_fire.read(1) - post_fire.read(2)) / (post_fire.read(1) + post_fire.read(2))

# Con estos calculamos el dNBR
dnbr = nbr_pre_fire - nbr_post_fire

# Guardamos el resultado en un nuevo archivo
profile = pre_fire.profile
profile.update(dtype=rasterio.float32, count=1)
# with rasterio.open("../rasters/lansat/dnbr.tif", "w", **profile) as dst:
#     dst.write(dnbr, 1)
    

In [11]:
# Ahora creamos categorias de valores para dNBR
dnbr_ranges = {
    'enhaced_regrowth_high': (-0.500, -0.251),
    'enhaced_regrowth_low': (-0.250, -0.101),
    'unburned': (-0.100, 0.99),
    'low_severity': (0.100, 0.269),
    'moderate_low_severity': (0.270, 0.439),
    'moderate_high_severity': (0.440, 0.659),
    'high_severity': (0.660, 1.300)
}

dnbr_counts = {
    'enhaced_regrowth_high': 0,
    'enhaced_regrowth_low': 0,
    'unburned': 0,
    'low_severity': 0,
    'moderate_low_severity': 0,
    'moderate_high_severity': 0,
    'high_severity': 0
}


In [13]:
# Recorremos la mascara dNBR y contamos cuantos pixeles hay en cada categoria
for key, value in dnbr_ranges.items():
    dnbr_counts[key] = ((dnbr >= value[0]) & (dnbr <= value[1])).sum()
    
dnbr_counts

{'enhaced_regrowth_high': 3208,
 'enhaced_regrowth_low': 94162,
 'unburned': 2525324,
 'low_severity': 6634,
 'moderate_low_severity': 1946,
 'moderate_high_severity': 231,
 'high_severity': 0}